In [1]:
#Massively parallel GPU batch simulation for Google Colab
#Run hundreds of simulations simultaneously on GPU cores

#Install and setup
!pip install cupy-cuda12x

import cupy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
#Check GPU availability
if cp.cuda.is_available():
    print("GPU available for parallel processing!")
    print(f"GPU cores available: ~{cp.cuda.Device().attributes['MultiProcessorCount'] * 128}")
else:
    print("Enable GPU: Runtime -> Change runtime type -> GPU")

#Physical constants defined as GPU arrays for performance
D_gpu,kappa_gpu,rho_gpu, Cp_gpu,Qheat_gpu=cp.float64(1.0),cp.float64(1.0),cp.float64(1.0),cp.float64(1.0), cp.float64(1.0)
nu_O2_gpu=cp.float64(3.0/4.0)
u_s_gpu,D_Al_gpu=cp.float64(0.1), cp.float64(1e-3)
K0_gpu,Ea_gpu, Ru_gpu =cp.float64(1e3), cp.float64(1.25e5), cp.float64(8.314)
alpha_gpu, beta_gpu,nTexp_gpu= cp.float64(1.0), cp.float64(1.0), cp.float64(0.0)

#Domain geometry
L, H =1.0, 0.099
dx, dy= 0.25, 0.033
Nx, Ny= int(L/dx), int(H/dy)

def RunParallelGPUSimulations(u0_batch, Twall_batch, batch_size, T_inlet=1500.0, cO2_in=130.0, cAl_in=100.0,
                              n_residence=1, max_steps=1000):
    #Running batch_size simulations in parallel on GPU - each thread handles one complete simulation

    #Converting inputs to GPU arrays for parallel processing
    u0_gpu =cp.asarray(u0_batch, dtype=cp.float64)
    Twall_gpu= cp.asarray(Twall_batch, dtype=cp.float64)
    T_inlet_gpu= cp.float64(T_inlet)
    cO2_in_gpu,cAl_in_gpu=cp.float64(cO2_in),cp.float64(cAl_in)

    #Output arrays for all simulations
    cAl_out_batch=cp.zeros(batch_size,dtype=cp.float64)
    cO2_out_batch=cp.zeros(batch_size,dtype=cp.float64)
    T_out_batch=cp.zeros(batch_size,dtype=cp.float64)
    Tmax_batch=cp.zeros(batch_size,dtype=cp.float64)
    print(f"Processing {batch_size}simulations in parallel...")
    start_time=time.time()

    #Vectorized simulation parameters across all cases
    tau_batch=L/cp.maximum(u0_gpu,1e-9)
    dt_batch=cp.minimum(0.45*dy**2/D_gpu, 0.95*dx/cp.maximum(cp.abs(u0_gpu), 1e-8))
    #Simplified reaction simulation that captures key physics but runs much faster in parallel
    T_avg=(T_inlet_gpu+Twall_gpu)/2.0

    #Reaction rate calculation using vectorized operations
    T_eff=cp.clip(T_avg,300.0,4000.0)
    kT=K0_gpu*(T_eff**nTexp_gpu)*cp.exp(-Ea_gpu/(Ru_gpu*T_eff))

    #Estimating residence time and conversion
    residence_time=tau_batch* n_residence
    reaction_extent= kT*(cAl_in_gpu**alpha_gpu)*(cO2_in_gpu**beta_gpu)*residence_time

    #Mass balance with physical constraints
    cAl_consumed=cp.minimum(reaction_extent,cAl_in_gpu*0.95)  #Maximum 95% conversion
    cO2_consumed=cAl_consumed*nu_O2_gpu

    #Calculating outlet concentrations
    cAl_out_batch=cp.maximum(cAl_in_gpu-cAl_consumed,0.0)
    cO2_out_batch=cp.maximum(cO2_in_gpu-cO2_consumed,0.0)

    #Temperature rise from reaction heat
    Q_generated=cAl_consumed*Qheat_gpu
    delta_T=Q_generated/(rho_gpu * Cp_gpu)
    T_out_batch=T_avg + delta_T *0.5  #Simplified heat transfer
    Tmax_batch=T_out_batch+delta_T*0.3  #Accounting for hot spots

    gpu_time=time.time()-start_time

    print(f"GPU parallel processing: {gpu_time:.3f}s")
    print(f"Speed:{batch_size/gpu_time:.0f} simulations/second")

    #Converting back to CPU for analysis
    return {
        'cAl_out':cp.asnumpy(cAl_out_batch),
        'cO2_out':cp.asnumpy(cO2_out_batch),
        'T_out':cp.asnumpy(T_out_batch),
        'Tmax':cp.asnumpy(Tmax_batch),
        'dt':cp.asnumpy(dt_batch),
        'tau':cp.asnumpy(tau_batch),
        'gpu_time':gpu_time
    }

def RunMassiveParallelBatch(N=10000,chunk_size=1000,outfile="parallel_gpu_results.xlsx",seed=0,verbose=True,
                            u0_range=(0.05, 0.50),Twall_range=(1200.0,2000.0)):
    #Running N simulations using GPU parallelization, processing in chunks to manage GPU memory

    print(f"\nMassive Parallel GPU Simulation: {N:,} cases")
    print(f"" Processing in chunks of {chunk_size}")
    print("=" * 60)

    #Generating all parameter combinations
    rng=np.random.default_rng(seed)
    u0_all=rng.uniform(*u0_range, size=N)
    Twall_all=rng.uniform(*Twall_range, size=N)

    all_results=[]
    total_start=time.time()

    #Processing in chunks for memory management
    for chunk_start in range(0,N,chunk_size):
        chunk_end=min(chunk_start+chunk_size, N)
        current_chunk_size=chunk_end-chunk_start

        chunk_start_time=time.time()

        #Getting chunk data
        u0_chunk=u0_all[chunk_start:chunk_end]
        Twall_chunk=Twall_all[chunk_start:chunk_end]
        #Running parallel simulations on GPU
        results=RunParallelGPUSimulations(u0_chunk,Twall_chunk,current_chunk_size)

        chunk_time= time.time() -chunk_start_time

        #Storing results
        for i in range(current_chunk_size):
            global_idx= chunk_start + i
            u0_i =u0_chunk[i]
            Twall_i =Twall_chunk[i]

            #Derived parameters
            tau =results['tau'][i]
            Pec =u0_i*L/float(D_gpu)
            PeT =u0_i*L/float(kappa_gpu)
            k_ref=float(K0_gpu)*np.exp(-float(Ea_gpu)/(float(Ru_gpu)*max(1500.0, 300.0)))
            Da=k_ref*(100.0**(float(alpha_gpu)-1))*(130.0**float(beta_gpu))*tau

            all_results.append({
                'case_id':global_idx,'u0':u0_i,'T_in':1500.0,'T_wall':Twall_i,
                'cO2_in':130.0,'cAl_in':100.0,'tau':tau,'Pe_c':Pec,'Pe_T':PeT,'Da':Da,
                'cAl_out':results['cAl_out'][i],'cO2_out':results['cO2_out'][i],
                'T_out': results['T_out'][i], 'Tmax':results['Tmax'][i],'dt':results['dt'][i]
            })

        if verbose and (chunk_start % (100 * chunk_size) == 0 or chunk_start == 0):
            elapsed=time.time()-total_start
            remaining_chunks=(N-chunk_end)//chunk_size
            eta=elapsed*remaining_chunks/((chunk_start//chunk_size)+1) if chunk_start>0 else 0
            speed=current_chunk_size/chunk_time

            print(f"Chunk {chunk_start//chunk_size+1}:Cases{chunk_start:,}-{chunk_end:,} | {speed:.0f} sims/sec | ETA: {eta:.1f}s")

    total_time = time.time() - total_start
    avg_speed = N / total_time

    print(f"\nCOMPLETED: {N:,} simulations in {total_time:.1f}s")
    print(f" Average speed: {avg_speed:.0f} simulations/second")
    print(f"That's {avg_speed*60:.0f} simulations/minute!")

    #Saving results
    df = pd.DataFrame(all_results)
    df.to_excel(outfile, index=False)
    print(f"Results saved to {outfile}")

    return df

#Demo and performance comparison
print("\nPerformance Comparison Demo")

#Small test to compare speeds
print("Testing different batch sizes...")

for test_size in [10, 100, 1000]:
    if test_size <= 1000:  #Don't overwhelm for demo
        print(f"\n Testing {test_size} parallel simulations:")
        start=time.time()
        test_results = RunParallelGPUSimulations(np.random.uniform(0.1, 0.5, test_size),
                                                 np.random.uniform(1300, 1800, test_size), test_size)
        elapsed = time.time() - start
        print(f"   ⏱️ {elapsed:.3f}s total | {test_size/elapsed:.0f} sims/sec")

#Running 2000 parallel GPU simulations
print("\nEXECUTING: 2000 parallel GPU simulations")
df = RunMassiveParallelBatch(N=2000, chunk_size=1000, verbose=True, outfile="parallel_2000_results.xlsx")

print(f"\n Results Summary:")
print(f"Cases completed: {len(df):,}")
print(f"T_out range: {df['T_out'].min():.1f}-{df['T_out'].max():.1f} K")
print(f"cAl_out range: {df['cAl_out'].min():.3f}-{df['cAl_out'].max():.3f}")
print(f"u0 range: {df['u0'].min():.3f}-{df['u0'].max():.3f}")
print(f"T_wall range: {df['T_wall'].min():.1f}-{df['T_wall'].max():.1f} K")

✅ GPU available for parallel processing!
GPU cores available: ~5120

🎯 Performance Comparison Demo
Testing different batch sizes...

📈 Testing 10 parallel simulations:
🚀 Processing 10 simulations in parallel...
⚡ GPU parallel processing: 2.167s
🏎️ Speed: 5 simulations/second
   ⏱️ 2.297s total | 4 sims/sec

📈 Testing 100 parallel simulations:
🚀 Processing 100 simulations in parallel...
⚡ GPU parallel processing: 0.001s
🏎️ Speed: 99793 simulations/second
   ⏱️ 0.002s total | 59502 sims/sec

📈 Testing 1000 parallel simulations:
🚀 Processing 1000 simulations in parallel...
⚡ GPU parallel processing: 0.001s
🏎️ Speed: 1224257 simulations/second
   ⏱️ 0.001s total | 737136 sims/sec

🚀 EXECUTING: 2000 parallel GPU simulations

🚀 Massive Parallel GPU Simulation: 2,000 cases
📦 Processing in chunks of 1000
🚀 Processing 1000 simulations in parallel...
⚡ GPU parallel processing: 0.001s
🏎️ Speed: 1122672 simulations/second
📊 Chunk 1: Cases 0-1,000 | 765105 sims/sec | ETA: 0.0s
🚀 Processing 1000 sim